# **Ajustar Anchura** 

Esta línea hace que se ajuste la anchura del notebook, por defecto la ajusta a un 92%

In [1]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.display import display, HTML
display(HTML("<style>.container{ width:92% }</style>"))

# **Descargar Dependencias**

Estos son los elementos que se tienen que descargar para un uso adecuado de todo el notebook. 

In [2]:
# !pip install swig
# !pip install wrds
# !pip install pyportfolioopt
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
# !pip install yfinance
# !pip install pandas_market_calendars

# **Se importan las librerías**

In [12]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# 2. Separar en entrenamiento y prueba
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# **Se descargan los datos históricos**

In [18]:
# IMPORTANTE: Las fechas deben seguir el formato 'año/mes/día' (YYYY-MM-DD)
# Estas fechas delimitan el periodo de tiempo del que se descargarán los datos históricos
START_DATE = '2024-01-01'   # Fecha de inicio del análisis
END_DATE = '2025-03-04'     # Fecha de fin del análisis

# Lista de símbolos (tickers) de las acciones que se analizarán
# Estos corresponden a empresas cotizadas en bolsa como Moderna, Nvidia, Uber, etc.
symbols = [
    "MRNA",  # Moderna Inc.
    "NVDA",  # Nvidia Corp.
    "UBER",  # Uber Technologies Inc.
    "ASML",  # ASML Holding N.V.
    "AMZN",  # Amazon.com Inc.
    "AAPL"   # Apple Inc.
]

# Usamos el módulo YahooDownloader de FinRL para descargar datos históricos de acciones
# Se especifica el rango de fechas y la lista de símbolos (acciones) definidos previamente
data = YahooDownloader(
    start_date = START_DATE,   # Fecha de inicio del periodo de análisis
    end_date = END_DATE,       # Fecha de fin del periodo de análisis
    ticker_list = symbols      # Lista de acciones a descargar (AAPL, AMZN, etc.)
).fetch_data()                 # Ejecuta la descarga y devuelve un DataFrame con los datos

# **Extracción de Indicadores Técnicos y Reconstrucción de la Estructura Temporal**

### 1. Extracción de Indicadores Técnicos

Después de descargar los datos históricos de precios para varias acciones, se aplica un proceso de **ingeniería de características** para enriquecer el conjunto de datos con variables útiles para el modelo de aprendizaje automatizado.

Para esto, se utiliza el módulo `FeatureEngineer` de la biblioteca FinRL. Este módulo permite calcular automáticamente varios **indicadores técnicos**, que son ampliamente utilizados en el análisis técnico del mercado bursátil. Estos indicadores ayudan a capturar tendencias, momentum y señales de sobrecompra o sobreventa en los precios.

Entre los indicadores extraídos se encuentran:

- **RSI (Relative Strength Index)**
- **MACD (Moving Average Convergence Divergence)**
- **Bollinger Bands**
- **Medias móviles (SMA, EMA)**
- **CCI, DX, y más**

Además, se incluyen variables adicionales como:

- **VIX**: índice de volatilidad implícita del mercado, útil para medir el "miedo" del mercado.
- **Turbulence**: una medida del comportamiento anómalo del mercado basada en desviaciones multivariadas.

Estos indicadores se calculan para cada acción de forma individual y se agregan como nuevas columnas al DataFrame resultante (`processed`).

---

### 2. Reconstrucción de la estructura fecha × acción

Una vez que se tienen los indicadores técnicos, se realiza un paso adicional: **reconstruir la estructura completa del conjunto de datos**, garantizando que todas las combinaciones posibles de fechas y acciones estén presentes.

#### ¿Por qué se hace esto?

En el mundo real, no todas las acciones tienen datos disponibles para todas las fechas (por ejemplo, por días festivos, suspensiones de cotización o errores en la descarga). Para asegurar que el conjunto de datos sea consistente y estructurado (especialmente útil para modelos temporales), se realiza lo siguiente:

- Se genera una lista completa de fechas entre la mínima y máxima fecha observada.
- Se toma la lista de acciones (tickers) presentes en el conjunto de datos.
- Se calcula el **producto cartesiano** de fechas × acciones, creando todas las combinaciones posibles.
- Este nuevo DataFrame se fusiona con los datos procesados originales para **rellenar los valores existentes** y dejar explícitos los faltantes.
- Finalmente, se filtran las fechas que realmente ocurrieron en el mercado para evitar incluir días como fines de semana o festivos.

Este paso garantiza que el conjunto de datos tenga una estructura rectangular y ordenada, lo cual es especialmente útil para la fase de modelado.

---


In [23]:
# Creamos un objeto FeatureEngineer para calcular automáticamente indicadores técnicos
fe = FeatureEngineer(
    use_technical_indicator=True,         # Activamos el cálculo de indicadores técnicos clásicos (RSI, MACD, etc.)
    tech_indicator_list=INDICATORS,       # Usamos la lista predefinida de indicadores de FinRL
    use_vix=True,                         # Incluye el índice VIX (volatilidad implícita del mercado)
    use_turbulence=True,                 # Incluye la medida de turbulencia financiera
    user_defined_feature=False           # No se agregan indicadores personalizados por ahora
)

# Aplicamos el preprocesamiento sobre el DataFrame descargado ('data') para generar nuevas columnas con indicadores
processed = fe.preprocess_data(data)

# --- Reconstruimos la estructura completa fecha × acción para evitar combinaciones faltantes ---

# Obtenemos la lista única de tickers (acciones)
list_ticker = processed["tic"].unique().tolist()

# Creamos una lista de fechas entre la mínima y máxima fecha disponibles en el dataset
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))

# Generamos todas las combinaciones posibles de (fecha, ticker)
combination = list(itertools.product(list_date, list_ticker))

# Creamos un nuevo DataFrame con todas las combinaciones posibles (fecha, acción)
# Luego hacemos un left join con los datos procesados para rellenar los datos existentes
processed_full = pd.DataFrame(combination, columns=["date", "tic"]).merge(
    processed, on=["date", "tic"], how="left"
)

# Filtramos para conservar solo las fechas que realmente estaban en los datos originales
# Esto evita que aparezcan fechas inexistentes (por ejemplo, fines de semana o días festivos)
processed_full = processed_full[processed_full['date'].isin(processed['date'])]

# Ordenamos los datos primero por 'tic' (símbolo de la acción) y luego por 'date'
# Esto es necesario para aplicar el rellenado hacia adelante (forward fill) correctamente dentro de cada acción
processed_full = processed_full.sort_values(['tic', 'date'])

# Rellenamos los valores faltantes con el último valor válido conocido hacia adelante (forward fill)
# Esto es útil porque algunos indicadores técnicos no tienen valor en los primeros días y así evitamos NaNs
processed_full = processed_full = processed_full.ffill()

# Eliminamos cualquier fila que aún tenga valores faltantes después del rellenado
# Esto suele ocurrir en los primeros días de cada acción, donde no hay valores previos para propagar
processed_full = processed_full.dropna()


#OPCIONAL: Visualizar la data
processed_full.head(5)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (291, 8)
Successfully added vix
Successfully added turbulence index


,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2024-01-02,AAPL,184.290405,187.070052,182.553128,185.789422,82488700.0,1.0,0.000000,185.551913,181.649015,0.0000,-66.666667,100.000000,184.290405,184.290405,13.20,0.0
6,2024-01-03,AAPL,182.910522,184.528677,182.096477,182.880742,58414500.0,2.0,-0.030959,185.551913,181.649015,0.0000,-66.666667,100.000000,183.600464,183.600464,14.04,0.0
12,2024-01-04,AAPL,180.587540,181.758954,179.565029,180.825785,71983600.0,3.0,-0.111483,186.338841,178.853471,0.0000,-100.000000,100.000000,182.596156,182.596156,14.13,0.0
18,2024-01-05,AAPL,179.862839,181.431354,178.860187,180.666963,62303300.0,4.0,-0.171540,186.012764,177.812889,0.0000,-77.623116,100.000000,181.912827,181.912827,13.35,0.0
36,2024-01-08,AAPL,184.210999,184.250716,180.180517,180.766224,59144500.0,0.0,-0.027540,186.475190,178.269732,51.3612,26.023162,7.073244,182.372461,182.372461,13.08,0.0


# **Cálculo y Etiquetado de la Volatilidad**

### Parte 1: Cálculo de la Volatilidad de 5 Días

La volatilidad es una medida de qué tanto varían los precios de una acción en un periodo de tiempo. En este caso, la calculamos como la **desviación estándar de los rendimientos diarios** en una ventana móvil de 5 días.

#### ¿Qué se hace?

- Se agrupan los datos por acción (`tic`), ya que la volatilidad debe calcularse de forma independiente para cada activo.
- Se aplica una **ventana móvil de 5 días** sobre la columna `return`, que representa el rendimiento diario.
- Dentro de esa ventana, se calcula la **desviación estándar**, lo cual nos da una estimación local de la volatilidad.
- El resultado se asigna como una nueva columna llamada `volatility_5d`.

#### ¿Por qué usar desviación estándar?

La desviación estándar es una medida clásica de **dispersión estadística**. Cuando los rendimientos de una acción fluctúan mucho en pocos días, la desviación estándar será alta. Por eso, se utiliza como una buena aproximación de la volatilidad en análisis financiero.

---

### Parte 2: Etiquetado de Días con "Alta Volatilidad"

Para usar modelos de clasificación, necesitamos transformar la volatilidad continua en una variable binaria. Lo hacemos creando una etiqueta que indique si un día tiene o no **alta volatilidad**.

#### ¿Qué se hace?

- Se agrupan los datos por acción (`tic`), ya que cada acción puede tener un nivel típico de volatilidad distinto.
- Se calcula el **percentil 75** (también llamado cuartil superior) de la columna `volatility_5d` para cada acción.
- Este valor actúa como un **umbral dinámico**: representa qué tan volátil debe ser un día para ser considerado "alto" en el contexto de esa acción.
- Para cada fila, se compara la volatilidad observada con ese umbral:
  - Si la volatilidad es mayor al percentil 75 → se etiqueta como `1` (alta volatilidad).
  - Si es menor o igual → se etiqueta como `0` (baja o normal volatilidad).
- El resultado se almacena en una nueva columna llamada `volatilidad_alta`.

---

#### ¿Qué es el percentil 75 y por qué se usa?

El percentil 75 es el valor por debajo del cual se encuentra el 75% de los datos. En este caso, representa un umbral de volatilidad "alta" relativo al comportamiento típico de cada acción. Si la volatilidad de un día supera este valor, se considera un evento inusualmente volátil. Esta estrategia permite adaptar el criterio de alta volatilidad a cada acción, en lugar de usar un valor fijo para todas.


In [26]:
# 1. Ordenamos por acción y fecha
processed_full = processed_full.sort_values(['tic', 'date'])

# 2. Calculamos el rendimiento diario por acción
processed_full['return'] = processed_full.groupby('tic')['close'].pct_change()

# 3. Calculamos la volatilidad como desviación estándar de 5 días sobre los rendimientos
processed_full['volatility_5d'] = processed_full.groupby('tic')['return'].rolling(5).std().reset_index(0, drop=True)

# 4. Etiquetamos los días con volatilidad alta (top 25% por acción)
def etiquetar_volatilidad(df, column='volatility_5d'):
    umbrales = df.groupby('tic')[column].transform(lambda x: x.quantile(0.75))
    df['volatilidad_alta'] = (df[column] > umbrales).astype(int)
    return df

processed_full = etiquetar_volatilidad(processed_full)

# 5. Eliminamos filas con valores faltantes
processed_full = processed_full.dropna()

processed_full

,date,tic,close,high,low,open,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,return,volatility_5d,volatilidad_alta
42,2024-01-09,AAPL,183.794052,183.803974,181.401569,182.582920,42841800.0,1.0,0.041713,186.458184,...,48.871272,29.331141,7.073244,182.609393,182.609393,12.760000,0.000000,-0.002263,0.014335,0
48,2024-01-10,AAPL,184.836426,185.044891,182.582924,183.009806,46792900.0,2.0,0.132628,186.823490,...,54.567469,76.254235,13.208570,182.927540,182.927540,12.690000,0.000000,0.005671,0.013924,0
54,2024-01-11,AAPL,184.240784,185.690176,182.285104,185.183874,49128400.0,3.0,0.163128,186.816261,...,51.195839,66.900271,21.476531,183.091696,183.091696,12.440000,0.000000,-0.003223,0.011889,0
60,2024-01-12,AAPL,184.568375,185.382421,183.843686,184.707356,40444700.0,4.0,0.198020,186.876197,...,52.853241,85.517404,21.476531,183.255771,183.255771,12.700000,0.000000,0.001778,0.011165,0
84,2024-01-16,AAPL,182.295029,182.920438,179.614645,180.835714,65603000.0,1.0,0.102792,186.626728,...,42.493474,-66.785229,29.102445,183.159697,183.159697,13.840000,0.000000,-0.012317,0.006729,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,2025-02-24,UBER,76.419998,78.879997,74.849998,78.650002,24368400.0,0.0,3.449048,86.061833,...,56.159591,65.676325,7.593468,71.628999,68.240166,18.980000,4.777818,-0.031309,0.023722,0
2525,2025-02-25,UBER,74.949997,76.370003,73.529999,76.360001,19559200.0,1.0,3.006083,86.140179,...,54.305930,39.701343,12.923284,71.928332,68.265666,19.430000,1.501099,-0.019236,0.014634,0
2531,2025-02-26,UBER,75.870003,76.489998,75.309998,75.330002,10328900.0,2.0,2.698164,86.198854,...,55.261977,47.506516,12.293895,72.267332,68.337499,19.100000,6.419113,0.012275,0.019213,0
2537,2025-02-27,UBER,74.209999,77.690002,73.709999,75.949997,22535900.0,3.0,2.293746,85.983300,...,53.184914,35.376200,18.748735,72.579666,68.380666,21.129999,9.438024,-0.021880,0.017570,0


# **Descripción de las columnas del dataset final (`data_para_modelo.csv`)**

A continuación se describen brevemente las columnas del conjunto de datos que se utilizará para entrenar los modelos de clasificación:

- **close**: Precio de cierre de la acción en el día correspondiente.
- **high**: Precio más alto alcanzado por la acción durante el día.
- **low**: Precio más bajo alcanzado por la acción durante el día.
- **open**: Precio de apertura de la acción en ese día.
- **volume**: Volumen de operaciones (cantidad de acciones intercambiadas en el día).
- **day**: Día de la semana representado como número (0 = lunes, 6 = domingo).

### Indicadores técnicos (features extraídas automáticamente):
- **macd**: Media móvil de convergencia/divergencia, indicador de momentum.
- **boll_ub** / **boll_lb**: Bandas de Bollinger superior e inferior, usadas para detectar sobrecompra o sobreventa.
- **rsi_30**: Índice de fuerza relativa (RSI) con ventana de 30 días.
- **cci_30**: Commodity Channel Index, mide la variación del precio respecto a su media.
- **dx_30**: Directional Movement Index, evalúa la fuerza de una tendencia.
- **close_30_sma** / **close_60_sma**: Medias móviles simples del precio de cierre en ventanas de 30 y 60 días.

### Etiqueta (target):
- **volatilidad_alta**: Variable binaria que indica si el día fue clasificado como de alta volatilidad (`1`) o no (`0`), calculado con base en el percentil 75 de la volatilidad histórica por acción.

---


In [27]:
# Opcional: inspeccionar columnas
print(processed_full.columns)

# Definir columnas a excluir
columnas_excluir = ['date', 'tic', 'return', 'volatility_5d']

# Crear DataFrame solo con features + target
df_model = processed_full.drop(columns=columnas_excluir, errors='ignore')

# Acomodar columnas para dejar 'volatilidad_alta' al final (opcional, por claridad)
columnas = [col for col in df_model.columns if col != 'volatilidad_alta'] + ['volatilidad_alta']
df_model = df_model[columnas]

Index(['date', 'tic', 'close', 'high', 'low', 'open', 'volume', 'day', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma', 'vix', 'turbulence', 'return', 'volatility_5d',
       'volatilidad_alta'],
      dtype='object')


In [28]:
df_model

,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,volatilidad_alta
42,183.794052,183.803974,181.401569,182.582920,42841800.0,1.0,0.041713,186.458184,178.760602,48.871272,29.331141,7.073244,182.609393,182.609393,12.760000,0.000000,0
48,184.836426,185.044891,182.582924,183.009806,46792900.0,2.0,0.132628,186.823490,179.031590,54.567469,76.254235,13.208570,182.927540,182.927540,12.690000,0.000000,0
54,184.240784,185.690176,182.285104,185.183874,49128400.0,3.0,0.163128,186.816261,179.367131,51.195839,66.900271,21.476531,183.091696,183.091696,12.440000,0.000000,0
60,184.568375,185.382421,183.843686,184.707356,40444700.0,4.0,0.198020,186.876197,179.635345,52.853241,85.517404,21.476531,183.255771,183.255771,12.700000,0.000000,0
84,182.295029,182.920438,179.614645,180.835714,65603000.0,1.0,0.102792,186.626728,179.692666,42.493474,-66.785229,29.102445,183.159697,183.159697,13.840000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,76.419998,78.879997,74.849998,78.650002,24368400.0,0.0,3.449048,86.061833,61.638165,56.159591,65.676325,7.593468,71.628999,68.240166,18.980000,4.777818,0
2525,74.949997,76.370003,73.529999,76.360001,19559200.0,1.0,3.006083,86.140179,62.177820,54.305930,39.701343,12.923284,71.928332,68.265666,19.430000,1.501099,0
2531,75.870003,76.489998,75.309998,75.330002,10328900.0,2.0,2.698164,86.198854,62.899145,55.261977,47.506516,12.293895,72.267332,68.337499,19.100000,6.419113,0
2537,74.209999,77.690002,73.709999,75.949997,22535900.0,3.0,2.293746,85.983300,63.860699,53.184914,35.376200,18.748735,72.579666,68.380666,21.129999,9.438024,0


## **Opcional**

Se guarda la data ya preprocesada

In [55]:
# df_model.to_csv("data_para_modelo.csv", index=False)
# print("Archivo 'data_para_modelo.csv' guardado correctamente.")

# **Escalarización de los datos**

In [56]:
# 1. Separar X (características) e y (etiqueta)
# Excluimos la columna 'volatilidad_alta' de X porque es la variable que queremos predecir
X = df_model.drop(columns=['volatilidad_alta'])
y = df_model['volatilidad_alta']

# 2. Dividir el conjunto en entrenamiento (80%) y prueba (20%)
# Usamos shuffle=False porque los datos son temporales (series de tiempo) y no deben mezclarse
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 3. Escalar las características
# Aplicamos StandardScaler para llevar todas las columnas numéricas a media 0 y desviación estándar 1
# Ajustamos el escalador con los datos de entrenamiento y luego transformamos ambos conjuntos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Reconstruir el DataFrame escalado completo (entrenamiento + prueba)
# Unimos ambos arrays escalados y los convertimos a un DataFrame con los mismos nombres de columnas originales
X_scaled = pd.DataFrame(
    np.concatenate([X_train_scaled, X_test_scaled]),
    columns=X.columns
)

# 5. Añadir la columna de etiqueta (volatilidad alta) al DataFrame escalado
# Concatenamos las etiquetas originales (y_train + y_test) y las alineamos con los datos escalados
X_scaled['volatilidad_alta'] = pd.concat([y_train, y_test]).reset_index(drop=True)

# 6. OPCIONAL: Guardar el DataFrame completo escalado como CSV
# X_scaled.to_csv("data_escalada.csv", index=False)
# print("Archivo 'data_escalada.csv' guardado correctamente.")

# **Análisis de la Distribución de Clases**

Antes de entrenar cualquier modelo, es importante revisar cuántos ejemplos pertenecen a cada clase en la variable objetivo `volatilidad_alta`. Para esto, se contabilizó el número y el porcentaje de ejemplos con:

- `0`: días de baja o normal volatilidad
- `1`: días de alta volatilidad

### ¿Por qué tiene sentido?

La etiqueta `volatilidad_alta` se construyó usando el **percentil 75** de la volatilidad histórica de cada acción. Esto significa que, por diseño, **aproximadamente el 25%** de los días más volátiles fueron etiquetados como `1` (alta volatilidad), y el **75% restante** como `0` (normal o baja volatilidad).

Esto explica que la distribución de clases no sea balanceada al 50%, sino más cercana a una proporción 75/25, lo cual es esperable y coherente con el criterio de etiquetado aplicado.

Esta revisión también permite tomar decisiones informadas sobre métricas de evaluación (como F1-score) y, si fuera necesario, aplicar técnicas de balanceo o ajuste de pesos durante el entrenamiento.


In [57]:
print("-"*50)
# Contar cuántas veces aparece cada clase (0 o 1) en la columna 'volatilidad_alta'
# Esto nos dice cuántos ejemplos hay de días con baja/normal volatilidad (0) y de alta volatilidad (1)
conteo_clases = X_scaled['volatilidad_alta'].value_counts()

# Mostramos el número total de ejemplos por clase
print(conteo_clases)

print("-"*50)
# Calculamos el porcentaje que representa cada clase respecto al total
# normalize=True hace que los resultados estén entre 0 y 1, y luego se multiplican por 100 para expresarlos como porcentaje
porcentaje_clases = X_scaled['volatilidad_alta'].value_counts(normalize=True) * 100

# Mostramos el porcentaje de cada clase (útil para entender el balance del dataset)
print(porcentaje_clases)
print("-"*50)

--------------------------------------------------
volatilidad_alta
0    1284
1     432
Name: count, dtype: int64
--------------------------------------------------
volatilidad_alta
0    74.825175
1    25.174825
Name: proportion, dtype: float64
--------------------------------------------------


# **Entrenamiento de los modelos**

## AWAAAAASS

La data ya fue preprocesada, escalada y dividida. Puedes empezar directamente a entrenar modelos de clasificación con los siguientes conjuntos:

- `X_train_scaled`: datos de entrenamiento escalados
- `X_test_scaled`: datos de prueba escalados
- `y_train`: etiquetas correspondientes a `X_train_scaled`
- `y_test`: etiquetas correspondientes a `X_test_scaled`

Todos los datos han sido escalados con `StandardScaler()` y ordenados sin mezclar (`shuffle=False`), respetando su naturaleza temporal.

Puedes usar estos conjuntos para probar modelos como:
- Random Forest
- Logistic Regression
- SVM
- Redes neuronales (MLP)

Eli si lees esto te quiero mucho, lo borras plox, xd

# **Random Forest** 

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Entrenar el modelo
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

# 2. Hacer predicciones sobre el conjunto de prueba
y_pred = clf.predict(X_test_scaled)

# 3. Evaluar el desempeño
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=["No Volátil", "Alta Volatilidad"]))

Matriz de confusión:
[[226  34]
 [ 56  28]]

Reporte de clasificación:
                  precision    recall  f1-score   support

      No Volátil       0.80      0.87      0.83       260
Alta Volatilidad       0.45      0.33      0.38        84

        accuracy                           0.74       344
       macro avg       0.63      0.60      0.61       344
    weighted avg       0.72      0.74      0.72       344

